<a href="https://colab.research.google.com/github/Sripathm2/UCLA_CS_245_Project5/blob/GNN/GNN/MPNN%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install spektral

In [151]:
import numpy as np
from scipy import sparse
import tensorflow as tf
import spektral
from spektral.layers.ops import sp_matrix_to_sp_tensor
from spektral.datasets.mnist import MNIST

data = MNIST()


class Net(tf.keras.Model):
    def __init__(self, window=6, dropout=.5, **kwargs):
        """
        Window: int. Window of days
        #LSTM hidden states: 64
        Training: 500 epocs, batchsize 8, Adam optimizer, LR 10-3
        """
        super().__init__(**kwargs)
        self._nets = self.build_MPNN_unit(dropout)
        self.permute = tf.keras.layers.Permute((2,1,3))
        self.flatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.LSTM1 = tf.keras.layers.LSTM(52, return_sequences=True)
        self.LSTM2 = tf.keras.layers.LSTM(52, return_sequences=False,
                                          return_state=True)

    def build_MPNN_unit(self, dropout):
        L1 = []
        L1.append(
            spektral.layers.MessagePassing(aggregate='sum',
                                           activation='relu')
            )
        L1.append(
            tf.keras.layers.BatchNormalization()
            )
        L1.append(
            tf.keras.layers.Dropout(dropout)
            )
        L2 = []
        L2.append(
            spektral.layers.MessagePassing(aggregate='sum',
                                           activation='relu')
            )
        L2.append(
            tf.keras.layers.BatchNormalization()
            )
        L2.append(
            tf.keras.layers.Dropout(dropout)
            )
        return (L1,L2)


    def run_MPNN_unit(self, Adj, X):
        L1, L2 = self._nets
        y = None
        for i in range(0,len(L1)):
            if i == 0: # MessagePassing layer
                y = L1[i].propagate(X, Adj)
                continue
            # print(i,L1[i])#, y)
            y = L1[i](y)
        H1 = y
        for i in range(0, len(L2)):
            if i == 0: # MessagePassing Layer
                y = L2[i].propagate(y, Adj)
                continue
            y = L2[i](y)
        H2 = y
        return tf.concat((H1,H2), axis=1)
    
    def call(self, Adj, X):
        H_list = []
        for i in range(Adj.shape[0]):
          a = sp_matrix_to_sp_tensor(Adj[i])
          H = self.run_MPNN_unit(a, X[i])
          H_list.append(H)
        H_out = tf.expand_dims(H_list, axis=0)
        print('H_out: ', H_out.shape)
        #LSTM_input = self.permute(H_out)[0]
        LSTM_input = self.flatten(H_out)
        print('LSTM Input: ',LSTM_input.shape)
        x = self.LSTM1(inputs=LSTM_input)
        print('After First LSTM: ',x.shape)
        x, final_memory_state, final_carry_state = self.LSTM2(inputs=x)
        print('After Second LSTM: ',x.shape)
        print('Feature matrix: ', X.shape)
        print('Output of LSTM: ',x)
        #x = X+x
        #Lin?
        #x = tf.keras.activations.relu(x)
        return x

# Create random Adj Matrices
A_list = []
for i in range(286):
  temp = np.zeros([52,52])
  for j in range(np.random.randint(1,25)):
    r = np.random.randint(0,52)
    s = np.random.randint(0,52)
    n = np.random.randn()
    temp[r,s] = n
    temp[s,r] = n
  A_list.append(temp)
Adj = tf.expand_dims(A_list, axis=0)[0]

# Create random node feature matrices
X = [np.random.rand(52, 5) for i in range(0,286)]
X = tf.expand_dims(X, axis=0)[0]

# Create random labels
y = np.random.rand(52, 1)


mod = Net(window=6)
mod(Adj, X)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

H_out:  (1, 286,

<tf.Tensor: shape=(1, 52), dtype=float32, numpy=
array([[-0.2356465 , -0.09460261, -0.08097994, -0.12809427, -0.09205884,
         0.01028476,  0.26044554, -0.10897244, -0.01098394, -0.04446738,
        -0.24701883, -0.10144805, -0.1559867 ,  0.06473605,  0.14410442,
         0.15812728,  0.33936325,  0.10017049,  0.02381019,  0.00832506,
        -0.14718044, -0.23101597,  0.00990979, -0.16445112,  0.24627207,
         0.19460455,  0.20630333, -0.07012433,  0.09746635,  0.07180098,
         0.11246537,  0.09626795,  0.24672973, -0.06638323, -0.08306702,
        -0.1877486 , -0.0822549 ,  0.00179241,  0.12709722,  0.03083797,
         0.2108809 ,  0.01050644, -0.02138388, -0.01025241, -0.00153005,
         0.10621783, -0.10933523, -0.10894186, -0.15470508,  0.23511297,
         0.2111    ,  0.19094522]], dtype=float32)>

In [108]:
a_list = []
for i in range(10):
  temp = np.ones([5,5])*i
  a_list.append(temp)
aaa = tf.expand_dims(a_list, axis=0)
print(aaa.shape.as_list)
print(np.random.randn())

<bound method TensorShape.as_list of TensorShape([1, 10, 5, 5])>
1.0518753575866997


In [128]:
print(aaa.shape)
print(aaa)
p = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
o = p(aaa)
print(o.shape)
print(o)

(1, 10, 5, 5)
tf.Tensor(
[[[[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[2. 2. 2. 2. 2.]
   [2. 2. 2. 2. 2.]
   [2. 2. 2. 2. 2.]
   [2. 2. 2. 2. 2.]
   [2. 2. 2. 2. 2.]]

  [[3. 3. 3. 3. 3.]
   [3. 3. 3. 3. 3.]
   [3. 3. 3. 3. 3.]
   [3. 3. 3. 3. 3.]
   [3. 3. 3. 3. 3.]]

  [[4. 4. 4. 4. 4.]
   [4. 4. 4. 4. 4.]
   [4. 4. 4. 4. 4.]
   [4. 4. 4. 4. 4.]
   [4. 4. 4. 4. 4.]]

  [[5. 5. 5. 5. 5.]
   [5. 5. 5. 5. 5.]
   [5. 5. 5. 5. 5.]
   [5. 5. 5. 5. 5.]
   [5. 5. 5. 5. 5.]]

  [[6. 6. 6. 6. 6.]
   [6. 6. 6. 6. 6.]
   [6. 6. 6. 6. 6.]
   [6. 6. 6. 6. 6.]
   [6. 6. 6. 6. 6.]]

  [[7. 7. 7. 7. 7.]
   [7. 7. 7. 7. 7.]
   [7. 7. 7. 7. 7.]
   [7. 7. 7. 7. 7.]
   [7. 7. 7. 7. 7.]]

  [[8. 8. 8. 8. 8.]
   [8. 8. 8. 8. 8.]
   [8. 8. 8. 8. 8.]
   [8. 8. 8. 8. 8.]
   [8. 8. 8. 8. 8.]]

  [[9. 9. 9. 9. 9.]
   [9. 9. 9. 9. 9.]
   [9. 9. 9. 9. 9